In [ ]:
# ---------------------------------------------
# Cell 1: Import required Python libraries
# ---------------------------------------------

# The 'os' module lets us work with the operating system:
# - check if files exist
# - build file paths
# - list files in directories, etc.
import os

# 'base64' is used to convert binary data (like images) into a text format.
# We’ll use this to encode images so they can be sent to the vision model.
import base64

# 'json' helps us work with JSON data (JavaScript Object Notation),
# which is a very common format for structured data and API responses.
import json

# 'pandas' is a powerful library for working with tabular data (rows and columns),
# like a spreadsheet in code. We'll use DataFrames to organize equipment and plans.
import pandas as pd

# ModelInference is the main class from the IBM watsonx.ai SDK
# that we use to call foundation models (LLMs, vision models, etc.).
# We will create:
#   - one ModelInference for the vision model (to analyze images)
#   - one ModelInference for the planner model (to generate workout plans)
from ibm_watsonx_ai.foundation_models import ModelInference


In [ ]:
# ---------------------------------------------
# Cell 2: Set your IBM Cloud credentials
# ---------------------------------------------
# This cell configures your access to IBM watsonx.ai.
# Each student must enter:
#   - the API key for the watsonx.ai service
#   - the Project ID (GUID) associated with your project in IBM Cloud
#
# IMPORTANT:
# - These values are specific to *your* IBM Cloud account.
# - Treat your API key like a password—never share it publicly.
# - If the instructor provides a shared key, keep it private.

credentials = {
    # The URL for your IBM Cloud region where watsonx.ai is deployed.
    # Most users will use "us-south", but other regions exist.
    "url": "https://us-south.ml.cloud.ibm.com",

    # Your IBM Cloud API key goes here.
    # You can generate one in the IBM Cloud console.
    "apikey": "Type your key here"   # ← students must replace this
}

# This is the identifier for the watsonx.ai *project* you created.
# It looks something like: "12345678-abcd-1234-abcd-123456abcdef"
# You can find it inside the watsonx.ai dashboard under "Manage Projects".
WATSONX_PROJECT_ID = "Enter Project ID here"     # ← students must replace this

# (Optional) Print a friendly reminder (without revealing anything sensitive)
print("Credentials and Project ID have been set. Ready to connect to watsonx.ai.")


In [ ]:
# -----------------------------------------------------------
# Cell 3: Create our two AI “agents” (models) from watsonx.ai
# -----------------------------------------------------------
# In this project we use TWO different AI models:
#
#   1. A VISION MODEL  → looks at images and describes equipment
#   2. A TEXT MODEL    → generates the workout plan based on
#                         user profile + detected equipment
#
# Both models are accessed using ModelInference, which sends
# requests to IBM’s watsonx.ai service using the credentials
# you defined in the previous cell.
#
# Students will get to see how multiple models can cooperate:
# Vision agent → Equipment descriptions → Planner agent → Final plan.
# -----------------------------------------------------------


# -----------------------------------------------------------
# Vision model: meta-llama/llama-3-2-11b-vision-instruct
# -----------------------------------------------------------
# This is a multimodal LLM:
#   - It can read TEXT
#   - It can also “see” IMAGES (via base64-encoded inputs)
#
# The vision model identifies what equipment is shown in the images
# (e.g., dumbbells, barbells, machines).
#
# Parameters:
#   - max_new_tokens: maximum size of the model's output
#   - temperature: 0.0 = fully deterministic (same output every time)
#     This is ideal for classroom labs.
vision_model = ModelInference(
    model_id="meta-llama/llama-3-2-11b-vision-instruct",
    credentials=credentials,            # API key + URL
    project_id=WATSONX_PROJECT_ID,      # which watsonx.ai project to use
    params={
        "max_new_tokens": 256,          # small output → faster + cheaper
        "temperature": 0.0              # repeatable output for students
    }
)


# -----------------------------------------------------------
# Planner model: ibm/granite-3-2-8b-instruct
# -----------------------------------------------------------
# This is a TEXT-ONLY model.
#
# It does NOT see images directly. Instead, it receives:
#   - the student's profile
#   - the equipment descriptions (generated by the vision model)
#
# It outputs a 1-week workout plan using a pipe-separated text table.
#
# Granite is chosen because:
#   - It’s deterministic (great for teaching)
#   - It handles structured text extremely well
#   - Very strong at following multi-step instructions
planner_model = ModelInference(
    model_id="ibm/granite-3-2-8b-instruct",
    credentials=credentials,
    project_id=WATSONX_PROJECT_ID,
    params={
        "max_new_tokens": 1024,         # larger output allowed (plans are long)
        "temperature": 0.0              # deterministic again
    }
)


# -----------------------------------------------------------
# Print confirmation — useful for debugging in class.
# -----------------------------------------------------------
print("Vision model loaded:", vision_model.model_id)
print("Planner model loaded:", planner_model.model_id)


In [ ]:
# -----------------------------------------------------------
# Cell 4: Load and encode gym equipment images
# -----------------------------------------------------------
# In this step, we prepare one or more images that show the
# gym equipment available to the user.
#
# The Vision Model cannot accept raw image files. It requires:
#   1) The image to be read as raw bytes (binary)
#   2) Then encoded as a base64 text string
#
# The base64 version is safe to send over the network inside
# the JSON request to the vision model.
#
# Students will update the file paths below to point to THEIR
# own images located on their computer.
# -----------------------------------------------------------


# 👇 Update these paths to point to the actual equipment images.
#    You can test with one image, then add two or three more.
#    If a file is missing, the code will safely raise a helpful error.
image_filepaths = [
    "/Users/armenpischdotchian/Desktop/equip1.png",
    "/Users/armenpischdotchian/Desktop/equip2.png",
    "/Users/armenpischdotchian/Desktop/equip3.png",
]

# These lists will store:
#   - base64-encoded versions of the images
#   - the corresponding filenames
images = []
filenames = []

# -----------------------------------------------------------
# Loop through each image path:
#   - Check if the file exists
#   - Read it in binary mode
#   - Convert ("encode") the bytes into a base64 string
#
# The base64 string can be transmitted to the vision model,
# which will decode it internally and visually interpret it.
# -----------------------------------------------------------
for path in image_filepaths:

    # 1. Verify that the file path exists on the system
    if not os.path.exists(path):
        raise FileNotFoundError(
            f"The file was not found: {path}\n"
            "→ Make sure your file paths are correct!"
        )

    # 2. Open the file in "rb" mode (rb = read binary)
    with open(path, "rb") as f:
        # Read all bytes, then convert to base64, then decode to UTF-8 text
        encoded = base64.b64encode(f.read()).decode("utf-8")

    # 3. Append the encoded image and its raw filename
    images.append(encoded)
    filenames.append(os.path.basename(path))

# -----------------------------------------------------------
# Print confirmation that the images were loaded successfully.
# This helps students debug file path issues early.
# -----------------------------------------------------------
print(f"Loaded {len(images)} image(s):", filenames)


In [ ]:
# -----------------------------------------------------------
# Cell 5: Vision Agent — Convert each image into structured info
# -----------------------------------------------------------
# In this cell we:
#   1. Define a helper function to build a multimodal prompt
#      (text + image) for the vision model.
#   2. Provide the vision model with instructions that require
#      *exactly one line* of structured output per image.
#   3. Run the vision model on each image.
#   4. Parse the returned text and store it in a Python list
#      as a structured dictionary.
#
# By the end, we will have a list called gym_equipment, where
# each item looks like:
# {
#     "filename": "equip1.png",
#     "description": "rack of dumbbells",
#     "category": "dumbbell",
#     "workout_type": "strength"
# }
# -----------------------------------------------------------


def build_vision_messages(prompt: str, image_b64: str):
    """
    Build a multimodal message for the vision model.

    The vision model expects:
      - A SYSTEM message giving it expert instructions.
      - A USER message containing:
            1) Some instructional text
            2) The image encoded as a base64 data URL

    The data URL format looks like:
      data:image/png;base64,<base64-data-here>

    Parameters
    ----------
    prompt : str
        The text instructions we want the model to follow.
    image_b64 : str
        The base64-encoded image (already prepared earlier).

    Returns
    -------
    list
        A list of message objects to be sent to the model.
    """

    # Convert raw base64 into a data-URL the model understands.
    data_url = f"data:image/png;base64,{image_b64}"

    return [
        # SYSTEM message sets the model's behavior / role
        {
            "role": "system",
            "content": (
                "You are a fitness assistant that analyzes images of gym equipment. "
                "You respond with a single line of text per image."
            ),
        },

        # USER message contains both text instructions AND the encoded image
        {
            "role": "user",
            "content": [
                {"type": "text", "text": prompt},
                {
                    "type": "image_url",
                    "image_url": {"url": data_url},
                },
            ],
        },
    ]


# -----------------------------------------------------------
# Vision model instructions:
# -----------------------------------------------------------
# We force the model to output EXACTLY one line using pipes.
# Example:
#   "rack of dumbbells | dumbbell | strength"
#
# This makes parsing 100x easier for beginners, and for agentic workflows.
# -----------------------------------------------------------

vision_prompt = """
Look at the image of gym equipment.

Respond with EXACTLY ONE LINE in this format:
description | category | workout_type

Where:
- description: short text, e.g. "rack of black dumbbells on a stand"
- category: ONE of [barbell, dumbbell, machine, bodyweight, other]
- workout_type: ONE of [strength, endurance, flexibility, balance, cardio, other]

Rules:
- No extra text, no explanations, no bullets.
- Do NOT use the '|' character except as the separator.
"""


# This list will hold the parsed equipment info from the vision model.
gym_equipment = []


# -----------------------------------------------------------
# Loop through each image:
#   - Send the image + prompt to the vision model
#   - Receive one-line structured text
#   - Split it using '|'
#   - Validate correct format
#   - Store in the gym_equipment list
# -----------------------------------------------------------
for i, image in enumerate(images):

    # Build the multimodal message (text + image)
    messages = build_vision_messages(vision_prompt, image)

    # Send to the vision model
    response = vision_model.chat(messages=messages)

    # Extract the raw text the model produced
    line = response["choices"][0]["message"]["content"].strip()

    print(f"\nImage {i+1}: {filenames[i]}")
    print("Vision output:", line)

    # Expect format: description | category | workout_type
    parts = [p.strip() for p in line.split("|")]

    if len(parts) != 3:
        raise ValueError(
            f"Unexpected vision output format for {filenames[i]}:\n{line}\n"
            "Expected: description | category | workout_type"
        )

    description, category, workout_type = parts

    # Store as a structured dictionary
    gym_equipment.append({
        "filename": filenames[i],
        "description": description,
        "category": category,
        "workout_type": workout_type,
    })


# -----------------------------------------------------------
# Final printout of all detected equipment in a readable JSON format
# -----------------------------------------------------------
print("\nDetected equipment:")
print(json.dumps(gym_equipment, indent=2))


In [ ]:
# -----------------------------------------------------------
# Cell 6: Convert detected equipment into a Pandas DataFrame
# -----------------------------------------------------------
# At this point, `gym_equipment` is a Python list of dictionaries.
# Example:
# [
#   {
#     "filename": "equip1.png",
#     "description": "rack of dumbbells",
#     "category": "dumbbell",
#     "workout_type": "strength"
#   },
#   ...
# ]
#
# Pandas DataFrames make it easy to:
#   - display structured data neatly
#   - sort, filter, and analyze information
#   - use tabular data to feed other agents or logic
#
# Here we convert our list of dicts → DataFrame and show it.
# -----------------------------------------------------------

# Convert the list of dictionaries to a DataFrame
df_equipment = pd.DataFrame(gym_equipment)

# Display the table visually in the notebook
# (In Jupyter, the last line of a cell is automatically rendered.)
df_equipment


In [ ]:
# -----------------------------------------------------------
# Cell 7: Define the USER PROFILE (who the plan is for)
# -----------------------------------------------------------
# The workout plan depends heavily on:
#   - age
#   - fitness level
#   - training experience
#   - personal goals
#   - available days per week
#   - injuries or limitations
#   - exercise preferences
#
# This block allows students to MODIFY these inputs and see how
# the planner model changes the weekly workout plan.
#
# This is an excellent place for experimentation:
#   - Change goals ("weight loss", "hypertrophy", "flexibility")
#   - Change availability (2 vs 5 days per week)
#   - Add injuries ("knee pain", "no overhead lifting")
#   - Try different preference sets
#
# The planner agent (Granite) will adapt its plan accordingly.
# -----------------------------------------------------------

user_profile = {
    "name": "Alex",                         # User's name (only for display)
    "age": 35,                              # Age can influence exercise intensity
    "gender": "unspecified",                # Optional metadata

    # Beginner / intermediate / advanced
    # This will affect volume, exercise difficulty, and selection.
    "fitness_level": "beginner",

    # Experience with lifting influences safety + complexity of movements.
    "experience_with_strength_training": "low",  # none / low / moderate / high

    # A list of goals; the model will integrate these into the plan structure.
    "primary_goals": [
        "build strength",
        "improve general fitness"
    ],

    # Number of workout days the plan should cover.
    "days_per_week_available": 3,

    # A realistic session length helps ensure the plan isn’t too long.
    "session_length_minutes": 45,

    # List any injuries—model avoids unsafe exercises when possible.
    "injuries_or_limitations": [],

    # Personal likes and dislikes guide exercise selection.
    "exercise_preferences": {
        "likes": ["free weights"],
        "dislikes": ["high impact jumping"]
    }
}

# -----------------------------------------------------------
# Convert both the user profile and equipment list into JSON
# -----------------------------------------------------------
# The planner model receives these values AS TEXT in the prompt.
# JSON formatting ensures:
#   - clean structure
#   - predictable formatting
#   - easier parsing for LLMs
# -----------------------------------------------------------

user_profile_json = json.dumps(user_profile, indent=2)
equipment_json = json.dumps(gym_equipment, indent=2)

# (Optional for debugging)
print("User profile (JSON):")
print(user_profile_json)

print("\nEquipment list (JSON):")
print(equipment_json)


In [ ]:
# -----------------------------------------------------------
# Cell 8: Build the Planner Prompt and Generate a Weekly Plan
# -----------------------------------------------------------
# In this cell, we:
#   1. Build a detailed prompt for the Granite planner model.
#   2. Include BOTH:
#        - the user profile (JSON)
#        - the available gym equipment (JSON)
#   3. Provide strict formatting rules so the model returns
#      a clean, pipe-separated table.
#   4. Send the prompt to the planner model.
#   5. Capture the raw workout plan text for parsing.
#
# WHY A PIPE-SEPARATED TABLE?
#   - It is easy for an LLM to produce.
#   - It is easy for Python to parse.
#   - It avoids JSON formatting errors that beginners struggle with.
#   - It forces the agent to structure its reasoning.
# -----------------------------------------------------------


# -----------------------------------------------------------
# Build the planner_prompt using an f-string so that the
# JSON versions of the user profile and equipment list are
# inserted directly into the prompt.
# -----------------------------------------------------------
planner_prompt = f"""
You are an experienced, safety-conscious personal trainer.

You will create a 1-week workout plan using ONLY the equipment from the equipment list plus bodyweight.

USER PROFILE (JSON):
{user_profile_json}

AVAILABLE EQUIPMENT (JSON LIST):
{equipment_json}

Plan constraints:
- Use days_per_week_available from the user profile.
- Each day: warmup, 3–5 main exercises, cooldown.
- Exercises must match available equipment categories (or bodyweight).
- Adjust volume based on fitness_level and experience_with_strength_training.
- Respect injuries_or_limitations and exercise_preferences.

Output format:
You MUST respond as a plain text table with '|' separators.
First line is the header, then one line per exercise.

Columns (in this exact order):
Day | Focus | Exercise | Equipment | Sets | RepsOrTime | TargetMuscle | Intensity | Warmup | Cooldown | DayNotes

Rules:
- Do not include any explanations, markdown, or extra text.
- Do not add or remove columns.
- Every line (after the header) must have exactly 11 fields separated by '|'.
- Do not use '|' inside any field value.
"""

# -----------------------------------------------------------
# Call the planner model:
#   The planner_model.chat() function sends the prompt to
#   Granite and returns the model’s output.
#
# Note:
#   - The "messages" structure follows the LLM chat format.
#   - We only need text here (no images).
# -----------------------------------------------------------
response = planner_model.chat(
    messages=[
        {
            "role": "user",
            "content": [{"type": "text", "text": planner_prompt}],
        }
    ]
)

# -----------------------------------------------------------
# Extract model output:
#   The model returns a dictionary with the plan inside the
#   "content" field of the first message choice.
# -----------------------------------------------------------
plan_text = response["choices"][0]["message"]["content"]

# Print the raw text so students can inspect the structure.
print("Raw Granite planner output:\n")
print(plan_text)


In [ ]:
# -----------------------------------------------------------
# Cell 9: Parse the planner's pipe-separated text into a table
# -----------------------------------------------------------
# Goal:
#   Convert the raw text output from the planner model (plan_text)
#   into a structured Pandas DataFrame (df_plan).
#
# The model was instructed to produce lines like:
#
#   Day | Focus | Exercise | Equipment | Sets | RepsOrTime | TargetMuscle | Intensity | Warmup | Cooldown | DayNotes
#   Day 1 | Full body strength | Goblet squat | dumbbell | 3 | 8–10 | legs | moderate | 5 min light cardio | stretching | Focus on good form
#
# Steps:
#   1. Split the text into lines.
#   2. Identify the header line (column names).
#   3. Parse each subsequent line as one exercise row.
#   4. Build a list of dictionaries, then convert to a DataFrame.
# -----------------------------------------------------------

# 1. Split the raw plan text into individual lines.
#    - strip() removes leading/trailing whitespace
#    - we keep only non-empty lines
lines = [l.strip() for l in plan_text.splitlines() if l.strip()]

# If there are no lines at all, something went wrong with the model output.
if not lines:
    raise ValueError("Planner output is empty.")

# 2. The first line is expected to be the header,
#    the remaining lines are data rows.
header_line = lines[0]
data_lines = lines[1:]

# These are the column names we EXPECT based on the prompt.
# The model was told to output columns in this exact order.
expected_cols = [
    "Day", "Focus", "Exercise", "Equipment", "Sets",
    "RepsOrTime", "TargetMuscle", "Intensity",
    "Warmup", "Cooldown", "DayNotes"
]

# 3. Parse the header line by splitting on the '|' separator.
header_parts = [p.strip() for p in header_line.split("|")]
print("Header line parsed as:", header_parts)

# (Optional teaching note)
# Here we could check that `header_parts` exactly matches `expected_cols`.
# For now, we just print it so students can visually verify it.


# This list will hold one dictionary per exercise row.
rows = []

# 4. Process each remaining line as a data row.
for line in data_lines:
    # Split the line into pieces using '|' and strip extra spaces.
    parts = [p.strip() for p in line.split("|")]

    # If the number of fields doesn't match the expected number of columns,
    # we consider the line malformed and skip it.
    # (This keeps the DataFrame clean, but may drop badly formatted lines.)
    if len(parts) != len(expected_cols):
        print("⚠️ Skipping malformed line:", line)
        continue

    # Zip together column names with the values for this row,
    # then turn that into a dictionary.
    row = dict(zip(expected_cols, parts))

    # Add the row dictionary to our list.
    rows.append(row)

# 5. Convert the list of row dicts into a Pandas DataFrame.
df_plan = pd.DataFrame(rows)

print("\n✅ Parsed workout plan into DataFrame:")
df_plan  # Display the table in the notebook


In [ ]:
# -----------------------------------------------------------
# Cell 10: Parse the workout plan → Create DataFrame → Build HTML report
# -----------------------------------------------------------
# This is the final “assembly” cell. It does FOUR major things:
#
# 1.  SAFELY parse the raw planner output (plan_text)
#     The model returns plain text, so we:
#        - split into lines
#        - detect the header row
#        - detect the exercise rows
#        - handle formatting mistakes
#
# 2.  Convert the structured rows into a Pandas DataFrame
#     This becomes df_plan, which is our final “clean table”.
#
# 3.  Build a readable HTML report with:
#        - a header block (user profile summary)
#        - a section for each day with an exercise table
#
# 4.  Save the final HTML file ("workout_plan.html") to disk
#     You can open it in any browser → Print → Save as PDF.
# -----------------------------------------------------------

import pandas as pd
import os

# -----------------------------
# 1) Parse planner output (plan_text) into df_plan
# -----------------------------

# Safety check: make sure the planner produced real text
if not isinstance(plan_text, str) or not plan_text.strip():
    raise ValueError(
        "plan_text is empty or not a string.\n"
        "Make sure the planner_model cell ran correctly and returned text."
    )

# Split output into individual non-empty lines
lines = [l.strip() for l in plan_text.splitlines() if l.strip()]

# Keep only lines that contain the pipe character '|'
# This ensures we ignore any accidental text the model might add.
table_lines = [l for l in lines if "|" in l]

if not table_lines:
    raise ValueError(
        "No table-like lines found in the plan_text.\n"
        "Check the raw planner output above."
    )

# The first line is the header with column names
header_line = table_lines[0]

# Everything after the header is a data row
data_lines = table_lines[1:]

# Expected column names (exact order)
expected_cols = [
    "Day", "Focus", "Exercise", "Equipment", "Sets",
    "RepsOrTime", "TargetMuscle", "Intensity",
    "Warmup", "Cooldown", "DayNotes"
]

# Show the parsed header so students can understand what’s going on
header_parts = [p.strip() for p in header_line.split("|")]
print("Parsed header:", header_parts)

rows = []

# -----------------------------
# Parse each line of the table
# -----------------------------
for line in data_lines:
    parts = [p.strip() for p in line.split("|")]

    # If too many columns → merge extras into the last one
    if len(parts) > len(expected_cols):
        parts = parts[:len(expected_cols)-1] + [" ".join(parts[len(expected_cols)-1:])]

    # If too few columns → pad with empty fields
    if len(parts) < len(expected_cols):
        parts += [""] * (len(expected_cols) - len(parts))

    # If any mismatch remains → skip the line
    if len(parts) != len(expected_cols):
        print("⚠️ Skipping malformed line:", line)
        continue

    # Build a dictionary: {"Day": "..", "Focus": "..", ...}
    row = dict(zip(expected_cols, parts))
    rows.append(row)

# Convert to DataFrame
df_plan = pd.DataFrame(rows)

# Remove completely empty rows (if any slipped in)
df_plan = df_plan.replace("", pd.NA).dropna(how="all").fillna("")

print("\n✅ Parsed workout plan into DataFrame:")
print(df_plan)
print("Shape:", df_plan.shape)

if df_plan.empty:
    raise ValueError("df_plan is empty. The plan text may not have followed the expected table format.")


# -----------------------------------------------------------
# 2) Build the HTML report (header + daily tables)
# -----------------------------------------------------------

# Remove accidental whitespace in column names
df_plan = df_plan.rename(columns=lambda c: c.strip() if isinstance(c, str) else c)

print("\nFinal df_plan columns:", list(df_plan.columns))

# Create a header block summarizing the user's profile
goals_text = ", ".join(user_profile.get("primary_goals", []))

header_html = f"""
<h1>Personalized Workout Plan</h1>
<p><strong>Name:</strong> {user_profile.get("name", "N/A")}</p>
<p><strong>Age:</strong> {user_profile.get("age", "N/A")}</p>
<p><strong>Gender:</strong> {user_profile.get("gender", "N/A")}</p>
<p><strong>Fitness Level:</strong> {user_profile.get("fitness_level", "N/A")}</p>
<p><strong>Goals:</strong> {goals_text}</p>
<p><strong>Days/Week Available:</strong> {user_profile.get("days_per_week_available", "N/A")}</p>
<p><strong>Session Length:</strong> {user_profile.get("session_length_minutes", "N/A")} minutes</p>
"""

if user_profile.get("injuries_or_limitations"):
    header_html += (
        f"<p><strong>Injuries / Limitations:</strong> "
        f"{', '.join(user_profile['injuries_or_limitations'])}</p>"
    )

# Start the HTML output
html_output = [header_html, "<hr>"]

# Columns to show in each day's exercise table
preferred_cols = [
    "Exercise", "Equipment", "Sets", "RepsOrTime", "TargetMuscle", "Intensity"
]

# Safety check: ensure we have a valid Day column
if "Day" not in df_plan.columns:
    df_plan["Day"] = "Plan"

# Loop through each day and build its section
for day_name in df_plan["Day"].unique():
    day_df = df_plan[df_plan["Day"] == day_name].copy()

    focus = day_df["Focus"].iloc[0] if "Focus" in day_df.columns else ""
    warmup = day_df["Warmup"].iloc[0] if "Warmup" in day_df.columns else ""
    cooldown = day_df["Cooldown"].iloc[0] if "Cooldown" in day_df.columns else ""
    notes = day_df["DayNotes"].iloc[0] if "DayNotes" in day_df.columns else ""

    # Keep only the columns we want to show (if they exist)
    display_cols = [c for c in preferred_cols if c in day_df.columns]

    html_output.append(f"<h2>{day_name}</h2>")
    
    if focus:
        html_output.append(f"<p><strong>Focus:</strong> {focus}</p>")
    if warmup:
        html_output.append(f"<p><strong>Warmup:</strong> {warmup}</p>")

    # Insert table of exercises
    if display_cols:
        html_output.append(day_df[display_cols].to_html(index=False, border=1))
    else:
        html_output.append(day_df.to_html(index=False, border=1))

    if cooldown:
        html_output.append(f"<p><strong>Cooldown:</strong> {cooldown}</p>")
    if notes:
        html_output.append(f"<p><strong>Notes:</strong> {notes}</p><br>")

# Join all blocks into a single HTML document
full_html = "\n".join(html_output)

# Save the HTML to disk
output_path = "workout_plan.html"
with open(output_path, "w", encoding="utf-8") as f:
    f.write(full_html)

print(f"\n✅ Saved HTML to: {os.path.abspath(output_path)}")
print("Open this file in a web browser, then go to: File → Print → Save as PDF")
